In [17]:
import os
import torch
import gdown
import random
import logging
import easydict
import numpy as np
from collections import Counter

In [2]:
# def init_logger():
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
logger = logging.getLogger(__name__)

In [4]:
args = easydict.EasyDict({
    "data_dir" : "C:/Users/USER/Desktop/workspace/practice/korean-ner/data",
    "data_name" : "train.tsv",
    "vocab_dir" : "C:/Users/USER/Desktop/workspace/practice/korean-ner/vocab",
    "wordvec_dir" : "C:/Users/USER/Desktop/workspace/test/korean-ner/wordvec",
    "label_file" : "label.txt",
    "w2v_file" : "word_vector_300d.vec",
    "word_emb_dim" : 300,
    "word_vocab_size" : 400000,
    "char_vocab_size" : 400000,
    "seed" : 7,

})

seed 설정

In [5]:
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)

데이터 불러오기

In [6]:
# def get_data(args):
with open(os.path.join(args.data_dir,args.data_name), 'r', encoding='utf-8') as f:
    document = f.readlines()
    # print(document)

build_vocab

In [7]:
# def build_vocab(args):

# load data & split sentence, ner_tag
sentences , ner_tags = [], []
with open(os.path.join(args.data_dir,args.data_name), 'r', encoding='utf-8') as f:
    document = f.readlines()
    for line in document:
        sentence , ner_tag = line.split('\t')
        sentences.append(sentence)
        ner_tags.append(ner_tag)

# total word, char list 만들기
total_word , total_char = [], [] 
for sentence in sentences:
    sentence = sentence.split()
    for word in sentence:
        for char in word:
            total_char.append(char)
        total_word.append(word)

# vocab_dir  만들기
if not os.path.exists(args.vocab_dir):
    os.makedirs(args.vocab_dir)

# build vocab (word, char)
word_vocab, char_vocab = [], []

word_vocab_path = os.path.join(args.vocab_dir, "word_vocab")
char_vocab_path = os.path.join(args.vocab_dir, "char_vocab")

word_counts = Counter(total_word)
word_vocab.append("PAD")
word_vocab.append("UNK")
word_vocab.extend([x[0] for x in word_counts.most_common()])
logger.info("Total word vocabulary size: {}".format(len(word_vocab)))

with open(word_vocab_path, 'w', encoding='utf-8') as f:
        for word in word_vocab:
            f.write(word + "\n")

char_counts = Counter(total_char)
char_vocab.append("PAD")
char_vocab.append("UNK")
char_vocab.extend([x[0] for x in char_counts.most_common()])
logger.info("Total char vocabulary size: {}".format(len(char_vocab)))

with open(char_vocab_path, 'w', encoding='utf-8') as f:
        for char in char_vocab:
            f.write(char + "\n")

# Set the exact vocab size
# If the original vocab size is smaller than args.vocab_size, then set args.vocab_size to original one
with open(word_vocab_path, 'r', encoding='utf-8') as f:
    word_list = f.readlines()
    args.word_vocab_size = min(len(word_list), args.word_vocab_size)

with open(char_vocab_path, 'r', encoding='utf-8') as f:
    char_list = f.readlines()
    args.char_vocab_size = min(len(char_list), args.char_vocab_size)

logger.info("args.word_vocab_size: {}".format(args.word_vocab_size))
logger.info("args.char_vocab_size: {}".format(args.char_vocab_size))


01/21/2022 16:42:36 - INFO - __main__ -   Total word vocabulary size: 307250
01/21/2022 16:42:36 - INFO - __main__ -   Total char vocabulary size: 2163
01/21/2022 16:42:36 - INFO - __main__ -   args.word_vocab_size: 307250
01/21/2022 16:42:36 - INFO - __main__ -   args.char_vocab_size: 2163


load_vocab

In [8]:
# def load_vocab(args):
word_vocab_path = os.path.join(args.vocab_dir, "word_vocab")
char_vocab_path = os.path.join(args.vocab_dir, "char_vocab")


if not os.path.exists(word_vocab_path):
    logger.warning("Please build word vocab first!!!")
    # return

if not os.path.exists(char_vocab_path):
    logger.warning("Please build char vocab first!!!")
    # return

word_vocab = {}
word_ids_to_tokens = []

#load word_vocab
with open(word_vocab_path,'r', encoding='utf-8') as f:
    word_list = f.readlines()
    args.word_vocab_size = min(len(word_list), args.word_vocab_size)

    for idx, word in enumerate(word_list[:args.word_vocab_size]):
        word = word.strip()
        word_vocab[word] = idx
        word_ids_to_tokens.append(word) 
        # print(word_ids_to_tokens)

char_vocab = {}
char_ids_to_tokens = []

#load_char_vocab
with open(char_vocab_path,'r', encoding='utf-8') as f:
    char_list = f.readlines()
    args.char_vocab_size = min(len(char_list), args.char_vocab_size)

    for idx, char in enumerate(char_list[:args.char_vocab_size]):
        char = char.strip()
        char_vocab[char] = idx
        char_ids_to_tokens.append(char) 

    # return word_vocab, word_ids_to_tokens, char_vocab, char_ids_to_tokens

get w2v

In [9]:
# def download_w2v(args):
# """ Download pretrained word vector """
w2v_path = os.path.join(args.wordvec_dir, args.w2v_file)
# Pretrained word vectors
if not os.path.exists(w2v_path):
    logger.info("Downloading pretrained word vectors...")
    gdown.download("https://drive.google.com/uc?id=1YX7yHm5MHZ-Icdm1ZX4X9_wD7UrXexJ-", w2v_path, quiet=False)

get label

In [ ]:
# def get_labels(args):
    # return [label.strip() for label in open(os.path.join(args.data_dir, args.label_file), 'r', encoding='utf-8')]
[label.strip() for label in open(os.path.join(args.data_dir, args.label_file), 'r', encoding='utf-8')]

load word matrix

In [13]:
# def load_word_matrix(args, word_vocab):
if not os.path.exists(args.wordvec_dir):
    os.makedirs(args.wordvec_dir)

 # Making new word vector (as list type)
logger.info("Building word matrix...")
embedding_index = dict()
with open(os.path.join(args.wordvec_dir, args.w2v_file), 'r', encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.rstrip().split(' ')
        embedding_index[tokens[0]] = list(map(float, tokens[1:]))

word_matrix = np.zeros((args.word_vocab_size, args.word_emb_dim), dtype='float32')
cnt = 0

for word, i in word_vocab.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        word_matrix[i] = np.array(embedding_vector, dtype='float32')
    else:
        word_matrix[i] = np.random.uniform(-0.25, 0.25, args.word_emb_dim)
        cnt += 1
logger.info(f'{cnt} words not in pretrained matrix')

word_matrix = torch.from_numpy(word_matrix)

    

01/21/2022 16:42:55 - INFO - __main__ -   Building word matrix...
01/21/2022 16:43:05 - INFO - __main__ -   170605 words not in pretrained matrix


In [14]:
word_matrix

tensor([[-0.0194, -0.1048, -0.0553,  ..., -0.0532,  0.0583,  0.0440],
        [ 0.1694,  0.0725, -0.0140,  ...,  0.0984,  0.1729, -0.0920],
        [-0.0219,  0.0241,  0.2438,  ..., -0.0234, -0.0474,  0.0429],
        ...,
        [-0.0749, -0.1652,  0.1287,  ...,  0.0107,  0.2355,  0.0500],
        [-0.0524,  0.0196,  0.0448,  ..., -0.0290,  0.0407, -0.0125],
        [ 0.1446,  0.0858,  0.1447,  ..., -0.1789,  0.0117, -0.2369]])